In [1]:
import pandas as pd
import numpy as np

##  1. 데이터 로드

In [2]:
selloutData = pd.read_csv("../lee/Python_ST_EX/dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [4]:
selloutData["NEW_QTY"]= np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])

In [5]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0


## 2. 53주차 제거

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

#### int형 YEARWEEK에서 100으로 나눈 몫은 YEAR이고, 100으로 나눈 나머지는 WEEK임.

In [7]:
int(201650/100)

2016

In [8]:
int(201650%100)

50

### YEAR에서 문제발생. 파이썬은 int형을 나눴는데 소숫점이 나오네?

####  해결방법1 . year를 강제형변환 다시해준다

In [9]:
selloutData["YEAR"] = (selloutData["YEARWEEK"] / 100).astype(int)
# 나누기 연산자 두개를 붙여 사용하면 강제형변환 안해도됨..

#### 해결방법2. // <==를 사용하면 몫만 출력된다

In [10]:
selloutData["WEEK"] = selloutData["YEARWEEK"] % 100

In [11]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


#### 해결방법3 . 함수를 만들어서 apply를 사용해서 해결한다.

In [12]:
def customFunction(inputColumn):
    yearValue = inputColumn / 100
    yearValue = int(yearValue)
    return yearValue

In [13]:
selloutData["YEAR"] = selloutData["YEARWEEK"].apply(customFunction)

In [14]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


In [15]:
refinedData = selloutData[selloutData.YEARWEEK % 100 < 53]

In [16]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


## 3. 데이터 정렬하기(rolling전에는 무조건 정렬해주어야함.)

In [17]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [18]:
sortedData = refinedData.sort_values(sortKey)

In [19]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,1
1757,A00,PRODUCT34,201402,679.0,679.0,2014,2
3125,A00,PRODUCT34,201403,578.0,578.0,2014,3
205,A00,PRODUCT34,201404,532.0,532.0,2014,4
4369,A00,PRODUCT34,201405,516.0,516.0,2014,5


In [20]:
#맨앞의 index번호 다시 재정렬 하기
indexData = sortedData.reset_index(drop=True)
#drop을 True로하면 원래 인덱스번호를 다지워 버린다.

In [21]:
indexData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,1
1,A00,PRODUCT34,201402,679.0,679.0,2014,2
2,A00,PRODUCT34,201403,578.0,578.0,2014,3
3,A00,PRODUCT34,201404,532.0,532.0,2014,4
4,A00,PRODUCT34,201405,516.0,516.0,2014,5


## 4.  rolling을 이용한 기본 이동평균 함수

In [22]:
indexData["MA"] = indexData.NEW_QTY.rolling(\
    window = 15, center = True, min_periods = 1).mean()
#min_periods = 1 을 사용하면 데이터 개수가 1개라도 있으면, mean을 구할수 있음.

In [23]:
indexData.to_csv("./result.csv")
#현재 경로에 CSV파일로 데이터 저장하기

##  5. groupby 복습

In [24]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [25]:
groupData = indexData.groupby(groupKey)[["NEW_QTY"]].mean()
groupData.head(5)

NEW_QTY
REGIONID PRODUCT   YEAR            
A00      PRODUCT34 2014  275.961538
                   2015   86.634615
                   2016   36.576923
         PRODUCT58 2014    2.673077
                   2015    5.711538

## 6. 조인!

In [26]:
groupData.head(5)

NEW_QTY
REGIONID PRODUCT   YEAR            
A00      PRODUCT34 2014  275.961538
                   2015   86.634615
                   2016   36.576923
         PRODUCT58 2014    2.673077
                   2015    5.711538

In [27]:
groupData1 = groupData.reset_index().head()

In [28]:
groupData1.columns = ["REGIONID","PRODUCT","YEAR","NEW_QTY_MEAN"]

In [29]:
groupData1

,REGIONID,PRODUCT,YEAR,NEW_QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


In [30]:
indexData.head(5)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000


In [31]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [32]:
groupKey2 = ["REGIONID","PRODUCT"]

In [33]:
mergeResult = pd.merge(indexData, groupData1,\
        left_on = groupKey, right_on = groupKey, how = "left")
# how = left or inner를통해 조인 종류를 선택가능 default값은 inner(아무것도 안쓰면) 
# inner 조인은 교집합, left 조인은 전부다 조인

#조인 키를 줄이게 되면 데이터가 뻥튀기 된다.

In [34]:
mergeResult.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000,275.961538


In [35]:
len(mergeResult)

123864

In [36]:
mergeResult.to_csv("./mergedData.csv")

## 조인한 데이터 다시 groupby하기 (함수를 만들어서)

## list에 그룹바이한것을 저장함.

### rolling으로 인한 오차(상품이 달라지는 부분에 대해서 구별하지 못하고 순차적으로 로직을 펼치기 때문에 오차가 발생함)를 최소화 하기 위해서 다시 groupby해서 데이터를 리스트에 담아서 따로 관리 조회

In [37]:
groupDataSet = mergeResult.groupby(groupKey2)

In [38]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])

In [39]:
mergeResult.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000,275.961538


In [40]:
oneGroup.head(5)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000,275.961538


### 위의 작업을 함수로 하면 리스트 주소 순서대로 아래에 쭉 다나온다

In [41]:
def groupRolling(oneGroup):

#   len(list(groupDataSet.groups))

#   oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])
#   oneGroup.head(5)

    indexGroupData = oneGroup.reset_index(drop = True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window = 5, center = True).mean()

    return indexGroupData

In [42]:
finalResult = mergeResult.groupby(groupKey2).apply(groupRolling)

In [43]:
a = finalResult.reset_index(drop = True) #drop = True는 index를 초기화 해주는 것이다.
a.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN,MA2
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.0,275.961538,NaN


In [44]:
finalResult.to_csv("./final2.csv",index = False)

##  7. 동일한 컬럼데이터 합치기

In [45]:
fullData = pd.read_csv("../lee/Python_ST_EX/dataset/kopo_channel_seasonality_new.csv")

In [46]:
a01Data = fullData[fullData.REGIONID == "A01"]
len(a01Data)

3611

In [47]:
a20Data = fullData[fullData.REGIONID == "A20"]
len(a20Data)

471

In [48]:
resultConcat = pd.concat([a01Data,a20Data], axis = 0)

In [49]:
resultConcat.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
47,A01,PRODUCT75,201403,23768.0
48,A01,PRODUCT8,201403,105.0
49,A01,PRODUCT19,201403,1513.0
50,A01,PRODUCT20,201403,59.0
51,A01,PRODUCT47,201403,4436.0


In [50]:
len(resultConcat)

4082

In [51]:
#컬럼헤더 한개만 바꾸는 법
# data1.rename(columns={"기존컬럼명":"바꿀컬럼명"})

### Quiz 지역별 상품의 데이터상 가장 최근 4주차의 NEW_QTY의 평균이 붙어서 나오게만들기

In [52]:
indexData1 = indexData[(indexData.YEAR == max(indexData.YEAR)) &
            (indexData.WEEK >= max(indexData.WEEK)- 3)]

In [53]:
indexData2 = indexData1.groupby(["REGIONID","PRODUCT"])[["NEW_QTY"]].mean()
indexData2.head()

NEW_QTY
REGIONID PRODUCT             
A00      PRODUCT34      37.25
         PRODUCT58      76.25
         PRODUCT59   28398.75
A01      PRODUCT1      390.75
         PRODUCT12  302132.00

In [54]:
indexData3 = indexData2.rename(columns={"NEW_QTY":"RECENT_QTY_AVG"})
indexData3 = indexData3.reset_index()

In [55]:
a.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN,MA2
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,275.961538,NaN
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,275.961538,NaN
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000,275.961538,593.2
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636,275.961538,545.6
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000,275.961538,491.2


In [56]:
mergeResult2 = pd.merge(indexData3, a,\
        left_on = groupKey2, right_on = groupKey2, how = "left")

In [57]:
mergeResult2.head()

,REGIONID,PRODUCT,RECENT_QTY_AVG,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA,NEW_QTY_MEAN,MA2
0,A00,PRODUCT34,37.25,201401,661.0,661.0,2014,1,520.000000,275.961538,NaN
1,A00,PRODUCT34,37.25,201402,679.0,679.0,2014,2,514.444444,275.961538,NaN
2,A00,PRODUCT34,37.25,201403,578.0,578.0,2014,3,516.800000,275.961538,593.2
3,A00,PRODUCT34,37.25,201404,532.0,532.0,2014,4,503.363636,275.961538,545.6
4,A00,PRODUCT34,37.25,201405,516.0,516.0,2014,5,490.750000,275.961538,491.2
